# Phase 1 : Compréhension du Métier et des Données

**Projet :** Optimisation Logistique Minière par Apprentissage par Renforcement

**Auteur :** Jean-Luc Mupasa

**Date :** Octobre 2025

---

### Objectif de ce Notebook

Ce document est la feuille de route de la Phase 1 de ma thèse. Il a pour but de :
1.  **Synthétiser la recherche bibliographique** pour définir l'état de l'art.
2.  **Définir les indicateurs de performance (KPIs)** qui mesureront le succès du projet.
3.  **Spécifier les exigences** de l'environnement de simulation à construire en Phase 2.

In [ ]:
# Imports pour l'analyse (à compléter si nécessaire)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Environnement prêt pour la Phase 1.")

## 1. Revue de Littérature (État de l'Art)

Cette section centralise les synthèses des articles clés qui définissent le problème du dispatching minier et les solutions existantes.

### Synthèse 1 : Alarie & Gamache (2002)

*   **Référence :** Alarie, S., & Gamache, M. (2002). Overview of Solution Strategies Used in Truck Dispatching Systems for Open Pit Mines. *International Journal of Surface Mining, Reclamation and Environment*, 16(1), 59-76.
*   **Problème Adressé :** Manque de vision globale et de structure dans les premières approches de dispatching.
*   **Méthode / Concepts :** Classifie les stratégies de dispatching en **single-stage** (heuristiques myopes, ex: assigner au plus proche) et **multi-stage** (planification + dispatch en temps réel). C'est le concept fondamental de DISPATCH.
*   **Résultat Principal :** Démontre la supériorité théorique de l'approche multi-étapes car elle introduit une vision de planification (l'"upper stage") qui guide les décisions en temps réel.
*   **Contribution à Mon Projet :** Cet article fournit le **cadre conceptuel** pour mon état de l'art. Il me permet de classer les systèmes existants et de positionner mon travail comme une amélioration de la stratégie multi-étapes.

### Synthèse 2 : White & Olson (1986)

*   **Référence :** White, J.W., & Olson, J.P. (1986). Computer-Based Dispatching in Mines with Concurrent Operating Objectives. *Mining Engineering*, 38(11), 1045-1054.
*   **Problème Adressé :** Comment assigner les camions pour maximiser la production tout en respectant des objectifs multiples (mélange de minerai, etc.).
*   **Méthode / Concepts :** C'est l'article fondateur de **DISPATCH**. Il propose un modèle en deux étapes : 
    1.  **Upper Stage (planification) :** Un modèle de Programmation Linéaire (LP) qui utilise des "pseudo-coûts" pour déterminer les flux de matériaux optimaux (tonnes/heure) pour chaque route.
    2.  **Lower Stage (temps réel) :** Une heuristique pour assigner les camions disponibles afin de respecter les flux définis par l'étape 1, en utilisant les concepts de `need-time` et `lost-tons`.
*   **Résultat Principal :** Création d'un système de dispatching robuste et largement adopté par l'industrie.
*   **Contribution à Mon Projet :** C'est la **description technique de la baseline** que je vais comparer à mon agent RL. Je vais implémenter une version simplifiée de cette logique.

### Synthèse 3 : Nazari et al. (2018)

*   **Référence :** Nazari, M., et al. (2018). Reinforcement Learning for Solving the Vehicle Routing Problem. *NeurIPS*.
*   **Problème Adressé :** Résoudre des problèmes de routage complexes (similaires au dispatching) avec des méthodes d'apprentissage.
*   **Méthode / Concepts :** Propose une architecture de Deep Reinforcement Learning (Pointer Network) pour résoudre le Problème de Tournée de Véhicules (VRP).
*   **Résultat Principal :** Montre que le RL peut atteindre des performances proches des solveurs exacts, tout en étant beaucoup plus rapide à l'exécution (après entraînement).
*   **Contribution à Mon Projet :** C'est ma **source d'inspiration technique principale**. Je vais m'inspirer de leur définition de l'état, de l'action et de la récompense, ainsi que de leur architecture de réseau de neurones, pour construire mon propre agent RL adapté au contexte minier.

*(... Dupliquez et complétez pour les 2-3 autres articles clés ...)*

## 2. Indicateurs de Performance Clés (KPIs)

Pour évaluer la performance de notre agent RL par rapport aux méthodes existantes, les KPIs suivants seront mesurés dans la simulation :

1.  **Consommation Totale de Carburant (Litres) :** L'objectif principal de l'optimisation. C'est la somme des consommations de chaque camion sur toute la durée de la simulation.
2.  **Production Totale (Tonnes) :** Quantité totale de minerai transportée au concasseur. Doit être maximisée.
3.  **Temps de Cycle Moyen (minutes) :** Temps moyen pour un camion de faire un cycle complet (aller à la pelle, charger, aller au concasseur, décharger, revenir). Un indicateur d'efficacité globale.
4.  **Taux d'Utilisation des Pelles (%) :** Pourcentage de temps où les pelles sont en train de charger, par opposition à attendre un camion. Un indicateur clé de la bonne synchronisation de la flotte.
5.  **Temps d'Attente Moyen par Camion (minutes) :** Temps moyen passé par un camion dans une file d'attente devant une pelle. C'est ce que notre agent RL cherchera à minimiser directement.

## 3. Spécifications de l'Environnement de Simulation

Le cahier des charges pour l'environnement `gymnasium` à construire en Phase 2 est le suivant :

- **Composants du Monde :**
    - **Nœuds :** Pelles (avec file d'attente), Concasseurs, Zones Tampons.
    - **Arcs :** Routes, définies par une distance, une pente et un état (ex: bon, dégradé).
    - **Agents :** Camions, définis par une capacité, une vitesse max, et un modèle de consommation.

- **Dynamique du Simulateur :**
    - L'environnement avancera par événements discrets (arrivée d'un camion, fin d'un chargement, etc.).
    - Des événements stochastiques seront introduits (ex: probabilité de dégradation d'une route).

- **Interface avec l'Agent RL :**
    - **État (Observation) :** L'agent RL devra recevoir à chaque étape un état du système (ex: position de tous les camions, longueur des files d'attente, état des routes).
    - **Action :** L'agent RL devra choisir une destination pour un camion qui vient de faire une demande.
    - **Récompense :** L'agent recevra une récompense négative basée sur le temps et le carburant consommés pour le trajet.